In [13]:
import dask.dataframe as dd
import pandas as pd
import csv

# Load data with error handling
ddf = dd.read_csv(
    '../data/complaints.csv',
    dtype=str,
    quoting=csv.QUOTE_NONE,
    on_bad_lines='skip',
    encoding='utf-8'
)

In [14]:
print(ddf.head())  # Sample
ddf.columns  # Check available columns
print("Number of rows:", ddf.shape[0].compute())
print("Number of columns:", ddf.shape[1])  

  Date received                                            Product  \
0    2025-06-20  Credit reporting or other personal consumer re...   
1    2025-06-20  Credit reporting or other personal consumer re...   
2    2025-06-20  Credit reporting or other personal consumer re...   
3    2025-06-20  Credit reporting or other personal consumer re...   
4    2025-06-20  Credit reporting or other personal consumer re...   

        Sub-product                                 Issue  \
0  Credit reporting  Incorrect information on your report   
1  Credit reporting           Improper use of your report   
2  Credit reporting  Incorrect information on your report   
3  Credit reporting           Improper use of your report   
4  Credit reporting           Improper use of your report   

                                       Sub-issue Consumer complaint narrative  \
0            Information belongs to someone else                         <NA>   
1  Reporting company used your report improperly  

In [15]:
print(ddf.columns.tolist())

['Date received', 'Product', 'Sub-product', 'Issue', 'Sub-issue', 'Consumer complaint narrative', 'Company public response', 'Company', 'State', 'ZIP code', 'Tags', 'Consumer consent provided?', 'Submitted via', 'Date sent to company', 'Company response to consumer', 'Timely response?', 'Consumer disputed?', 'Complaint ID']


In [21]:
# Define a function to map messy product labels into 5 target categories
def standardize_product(product):
    if pd.isnull(product):
        return None
    product = product.lower()
    if "credit card" in product:
        return "Credit Card"
    elif "personal loan" in product or "consumer loan" in product:
        return "Personal Loan"
    elif "buy now" in product or "bnpl" in product:
        return "Buy Now, Pay Later (BNPL)"
    elif "savings" in product or "checking" in product or "bank account" in product:
        return "Savings Account"
    elif "money transfer" in product:
        return "Money Transfers"
    else:
        return None

# Apply the mapping
ddf['Standardized Product'] = ddf['Product'].map(standardize_product, meta=('Standardized Product', 'str'))

# Step 3: Filter for only valid product categories and non-empty narratives
# This handles both null and empty/whitespace complaints
filtered_ddf = ddf[
    ddf['Standardized Product'].notnull() &
    ddf['Consumer complaint narrative'].map(lambda x: bool(str(x).strip()) if pd.notnull(x) else False, meta=('Consumer complaint narrative', 'bool'))
]

# Step 4: Add narrative length column
filtered_ddf['Narrative Length'] = filtered_ddf['Consumer complaint narrative'].map(
    lambda x: len(str(x).split()),
    meta=('Narrative Length', 'int')
)

# Step 5: Count complaints per product
product_counts = filtered_ddf['Standardized Product'].value_counts().compute()
print("Complaint counts per product category:\n", product_counts)

# Step 6: Summary stats for narrative length
narrative_stats = filtered_ddf['Narrative Length'].describe().compute()
print("\nNarrative length statistics:\n", narrative_stats)

# Step 7: Count of complaints with/without narratives
with_narrative = ddf['Consumer complaint narrative'].map(lambda x: bool(str(x).strip()) if pd.notnull(x) else False, meta=('x', 'bool')).sum().compute()
total = len(ddf)
print(f"\nComplaints with non-empty narrative: {with_narrative}")
print(f"Complaints without narrative or only whitespace: {total - with_narrative}")

Complaint counts per product category:
 Standardized Product
Savings Account               80935
Credit Card                  108726
Buy Now, Pay Later (BNPL)         9
Money Transfers               30996
Personal Loan                  3102
Name: count, dtype: int64

Narrative length statistics:
 count    223768.000000
mean         28.373355
std          49.346518
min           1.000000
25%           8.000000
50%          17.000000
75%          52.000000
max        2411.000000
Name: Narrative Length, dtype: float64

Complaints with non-empty narrative: 4192454
Complaints without narrative or only whitespace: 6312435


In [19]:
print(ddf.columns.tolist())

['Date received', 'Product', 'Sub-product', 'Issue', 'Sub-issue', 'Consumer complaint narrative', 'Company public response', 'Company', 'State', 'ZIP code', 'Tags', 'Consumer consent provided?', 'Submitted via', 'Date sent to company', 'Company response to consumer', 'Timely response?', 'Consumer disputed?', 'Complaint ID', 'Standardized Product', 'Narrative Length']


In [25]:
import re

# Define text cleaning function
def clean_text(text):
    if pd.isnull(text):
        return ""
    
    # Lowercase
    text = text.lower()
    
    # Remove boilerplate phrases (customizable)
    boilerplate_phrases = [
        r"i am writing to (file|submit) a complaint",
        r"i would like to complain about",
        r"this complaint is regarding",
        r"dear.*?consumer financial protection bureau",  
    ]
    for phrase in boilerplate_phrases:
        text = re.sub(phrase, "", text)
    
    # Remove special characters (except basic punctuation)
    text = re.sub(r"[^a-zA-Z0-9\s.,!?]", " ", text)
    
    # Remove extra spaces
    text = re.sub(r"\s+", " ", text).strip()
    
    return text

# Add cleaned version of narratives
filtered_ddf['Cleaned Narrative'] = filtered_ddf['Consumer complaint narrative'].map(
    clean_text, meta=('Cleaned Narrative', 'str')
)

print("Number of rows:", filtered_ddf.shape[0].compute())
print("Number of columns:", filtered_ddf.shape[1])  


# Save filtered and cleaned dataset (best format)
filtered_ddf.to_parquet("../data/filtered_complaints.parquet", engine="pyarrow", write_index=False)



Number of rows: 223768
Number of columns: 21


In [27]:
# Convert to Pandas
filtered_df = filtered_ddf.compute()

# Save to CSV
filtered_df.to_csv("../data/filtered_complaints.csv", index=False)
